In [95]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
from scipy.spatial import cKDTree

# 1. 격자 구조 인구밀도 데이터 -> 중심점 계산
def calculate_centroids(grid_df):
    grid_df['centroid'] = grid_df.geometry.centroid
    return grid_df

# 2. 좌표 체계 맞추기
def align_crs(grid_df, shops_gdf):
    # 격자 데이터 CRS 설정 (EPSG:5181 사용)
    if grid_df.crs is None:
        grid_df = grid_df.set_crs(epsg=5181, allow_override=True)
    
    # 상점 데이터 CRS 설정 (EPSG:5181 사용)
    if shops_gdf.crs is None:
        shops_gdf = shops_gdf.set_crs(epsg=5181, allow_override=True)
    
    # 상점 데이터를 격자 데이터의 CRS로 변환
    shops_gdf = shops_gdf.to_crs(grid_df.crs)
    
    return grid_df, shops_gdf


# 3. 격자 중심점과 상점 위치 간의 거리 계산
def calculate_accessibility(grid_df, shops_gdf, buffer_distance):
    grid_df['accessibility'] = 0
    
    # grid_df 중심점과 shops_gdf의 좌표를 numpy 배열로 변환
    centroids = np.array([[point.x, point.y] for point in grid_df.centroid])
    shop_coords = np.array([[point.x, point.y] for point in shops_gdf.geometry])
    
    print(f"Centroids shape: {centroids.shape}")
    print(f"Shop coordinates shape: {shop_coords.shape}")
    
    # array가 비어있는지 확인
    if centroids.shape[0] == 0 or shop_coords.shape[0] == 0:
        raise ValueError("Centroid or shop coordinates array is empty")
    
    # KD-Tree를 사용하여 상점 위치에 대한 거리 계산
    tree = cKDTree(shop_coords)
    
    # 각각의 인구 밀도 중심점에 대해, 근접한 상점 개수 계산
    for centroid, index in zip(centroids, grid_df.index):
        indices = tree.query_ball_point(centroid, buffer_distance)
        grid_df.at[index, 'accessibility'] = len(indices)
    
    return grid_df

# 데이터 로드 및 처리 예시
grid_file = 'data/서울시 동대문구/(B100)국토통계_인구정보-총 인구 수(전체)-(격자) 100M_서울특별시 동대문구_202404/nlsp_021001001.shp'  # 격자 데이터 파일 경로
shops_file = 'data/서울시 동대문구/동대문구상권정보.shp'  # 상점 데이터 파일 경로
buffer_distance = 500  # 분석에 사용할 버퍼 거리 (단위: 미터)

# 격자 및 상점 데이터 불러오기
grid_df = gpd.read_file(grid_file)
shops_gdf = gpd.read_file(shops_file, encoding='ISO-8859-1')

# 좌표 체계 맞추기
grid_df, shops_gdf = align_crs(grid_df, shops_gdf)

# 격자 중심점 계산
grid_df = calculate_centroids(grid_df)

# 접근성 분석
grid_df = calculate_accessibility(grid_df, shops_gdf, buffer_distance)

# 결과 출력
print(grid_df.head())

Centroids shape: (1539, 2)
Shop coordinates shape: (25, 2)
            gid      lbl     val  \
0  ë¤ì¬609556   367.00   367.0   
1  ë¤ì¬624528  1144.00  1144.0   
2  ë¤ì¬625528  1369.00  1369.0   
3  ë¤ì¬616520   225.00   225.0   
4  ë¤ì¬615519   484.00   484.0   

                                            geometry                centroid  \
0  POLYGON ((960900 1955600, 960900 1955700, 9610...  POINT (960950 1955650)   
1  POLYGON ((962400 1952800, 962400 1952900, 9625...  POINT (962450 1952850)   
2  POLYGON ((962500 1952800, 962500 1952900, 9626...  POINT (962550 1952850)   
3  POLYGON ((961600 1952000, 961600 1952100, 9617...  POINT (961650 1952050)   
4  POLYGON ((961500 1951900, 961500 1952000, 9616...  POINT (961550 1951950)   

   accessibility  
0              0  
1              2  
2              2  
3              0  
4              0  


In [97]:
# 4. 상점별 PPR 값 계산 (2SFCA 기법 - 1단계)
def calculate_ppr_for_shops(grid_df, shops_gdf, buffer_distance):
    shops_gdf['PPR'] = 0
    
    # grid_df 중심점과 shops_gdf의 좌표를 numpy 배열로 변환
    centroids = np.array([[point.x, point.y] for point in grid_df.centroid])
    shop_coords = np.array([[point.x, point.y] for point in shops_gdf.geometry])
    
    # KD-Tree를 사용하여 인구 위치에 대한 거리 계산
    tree = cKDTree(centroids)
    
    # 각각의 상점 위치에 대해, 근접한 격자의 인구 밀도 합계 계산
    for shop_idx, shop in shops_gdf.iterrows():
        indices = tree.query_ball_point([shop.geometry.x, shop.geometry.y], buffer_distance)
        ppr_value = grid_df.iloc[indices]['val'].sum()
        shops_gdf.at[shop_idx, 'PPR'] = 1/ppr_value
    
    return shops_gdf

shops_gdf = calculate_ppr_for_shops(grid_df, shops_gdf, buffer_distance)

print(shops_gdf['PPR'])
print(shops_gdf.shape)

0     0.000029
1     0.000029
2     0.000056
3     0.000057
4     0.000047
5     0.000056
6     0.000151
7     0.000048
8     0.000037
9     0.000042
10    0.000125
11    0.000059
12    0.000060
13    0.000049
14    0.000047
15    0.000059
16    0.000035
17    0.000065
18    0.000048
19    0.000048
20    0.000070
21    0.000035
22    0.000069
23    0.000047
24    0.000042
Name: PPR, dtype: float64
(25, 28)


/tmp/ipykernel_935945/1796636700.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.9391018104867154e-05' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  shops_gdf.at[shop_idx, 'PPR'] = 1/ppr_value


In [93]:
# 5. 격자별 접근성 지수 계산 (2SFCA 기법 - 2단계)
def calculate_accessibility_for_grids(grid_df, shops_gdf, buffer_distance):
    grid_df['accessibility'] = 0
    
    # grid_df 중심점과 shops_gdf의 좌표를 numpy 배열로 변환
    centroids = np.array([[point.x, point.y] for point in grid_df.centroid])
    shop_coords = np.array([[point.x, point.y] for point in shops_gdf.geometry])
    shop_ppr_values = shops_gdf['PPR'].values
    
    # KD-Tree를 사용하여 상점 위치에 대한 거리 계산
    tree = cKDTree(shop_coords)
    
    # 각각의 인구 밀도(중심점)에 대해, buffer distance 내에 있는 상점들의 PPR 합계 계산
    for grid_idx, centroid in enumerate(centroids):
        indices = tree.query_ball_point(centroid, buffer_distance)
        accessibility_value = shop_ppr_values[indices].sum()
        grid_df.at[grid_idx, 'accessibility'] = accessibility_value
    
    return grid_df

# 격자별 접근성 지수 계산
grid_df = calculate_accessibility_for_grids(grid_df, shops_gdf, buffer_distance)

print(grid_df.head())

            gid      lbl     val  \
0  ë¤ì¬609556   367.00   367.0   
1  ë¤ì¬624528  1144.00  1144.0   
2  ë¤ì¬625528  1369.00  1369.0   
3  ë¤ì¬616520   225.00   225.0   
4  ë¤ì¬615519   484.00   484.0   

                                            geometry                centroid  \
0  POLYGON ((960900 1955600, 960900 1955700, 9610...  POINT (960950 1955650)   
1  POLYGON ((962400 1952800, 962400 1952900, 9625...  POINT (962450 1952850)   
2  POLYGON ((962500 1952800, 962500 1952900, 9626...  POINT (962550 1952850)   
3  POLYGON ((961600 1952000, 961600 1952100, 9617...  POINT (961650 1952050)   
4  POLYGON ((961500 1951900, 961500 1952000, 9616...  POINT (961550 1951950)   

   accessibility  
0       0.000000  
1       0.000059  
2       0.000059  
3       0.000000  
4       0.000000  


/tmp/ipykernel_935945/1450825006.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.878203620973431e-05' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  grid_df.at[grid_idx, 'accessibility'] = accessibility_value
